In [57]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint

In [58]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Osama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [60]:
train_classification_df.groupby("category").count()

,user_id
category,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [61]:
username2_category["kod8net"]

'tech'

In [62]:
'''train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]




# Load the additional CSV file
additional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"
additional_data_df = pd.read_csv(additional_data_path)

# Extract and rename the relevant columns
additional_data_df = additional_data_df[['Unnamed: 0', 'influencerCategory']].rename(columns={
    'Unnamed: 0': 'user_id', 
    'influencerCategory': 'category'
})

additional_data_df = additional_data_df.dropna()



# Convert the 'category' column to string type and apply .lower(), handling any NaN or unexpected values
additional_data_df['category'] = additional_data_df['category'].astype(str).fillna('').apply(str.lower)

# Append the new data to the original train_classification_df
train_classification_df = pd.concat([train_classification_df, additional_data_df], ignore_index=True)

# Update the username2_category dictionary with the new data
username2_category.update(additional_data_df.set_index("user_id").to_dict()["category"])

# Check the updated data by viewing the first few rows
print(train_classification_df.head())

# Re-check the category distribution
train_classification_df.groupby("category").count()'''


'train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)\ntrain_classification_df = train_classification_df.rename(columns={\'Unnamed: 0\': \'user_id\', \'label\': \'category\'})\n\n# Unifying labels\ntrain_classification_df["category"] = train_classification_df["category"].apply(str.lower)\nusername2_category = train_classification_df.set_index("user_id").to_dict()["category"]\n\n\n\n\n# Load the additional CSV file\nadditional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"\nadditional_data_df = pd.read_csv(additional_data_path)\n\n# Extract and rename the relevant columns\nadditional_data_df = additional_data_df[[\'Unnamed: 0\', \'influencerCategory\']].rename(columns={\n    \'Unnamed: 0\': \'user_id\', \n    \'influencerCategory\': \'category\'\n})\n\nadditional_data_df = additional_data_df.dropna()\n\n\n\n# Convert the \'category\' column to string type and apply .lower(), handling any NaN or unexpec

In [63]:
train_data_path = "/Users/Osama/Downloads/CS412PROJ/training-dataset.jsonl.gz"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with gzip.open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile

In [64]:
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.shape

(2741, 44)

In [65]:
test_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji


def preprocess_text(text: str):
    

    # lower casing Turkish Text, Don't use str.lower :)
    text = text.casefold()

    #text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and punctuation
    # HERE THE EMOJIS stuff are being removed, you may want to keep them :D
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


corpus = []

# to keep the label order
train_usernames = []

for username, posts in username2posts_train.items():
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)


  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)



#custom_stopwords = list(set(turkish_stopwords).union({
 #   'the', 'and', 'with', 'for', 'you', 'to', 'of', 'in', 'our', 'your', 'is', 'are','bir'
#}))
vectorizer = TfidfVectorizer(stop_words=turkish_stopwords, max_features=10000,min_df=10,ngram_range=(1, 3))

# fit the vectorizer
vectorizer.fit(corpus)

# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]
feature_names = vectorizer.get_feature_names_out()

# Inspect the frequency of each word
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)

# Show the most frequent words (words in many posts)
print(df_tfidf.sum().sort_values(ascending=False).head(30))




test_usernames = []
test_corpus = []
for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

bir            192.474290
the            106.220712
and             65.960927
to              59.345598
of              59.225437
olsun           56.723444
yeni            46.531255
istanbul        46.083638
in              44.908546
olarak          43.520030
olan            41.210234
with            39.918309
kutlu           39.892111
kutlu olsun     39.315072
iyi             37.609096
güzel           37.207437
kadar           36.265992
you             35.986349
for             34.862311
özel            32.566009
bilgi           32.489645
devam           32.122644
var             31.813758
our             31.544601
kasım           31.361084
günü            31.188695
birlikte        30.770312
büyük           30.195825
ilk             29.995598
teşekkür        29.513940
dtype: float64


In [67]:
# Making sure everything is fine
assert y_train.count("NA") == 0


In [68]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['ab', 'abant', 'abd', ..., 'şıklık', 'şıklığı', 'şımartın'],
      dtype=object)

In [69]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
df_tfidf.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,abiye,abone,about,about the,...,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şık bir,şıklık,şıklığı,şımartın
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.042156,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [70]:
df_tfidf.shape

(2741, 10000)

In [71]:
print("Shape of df_tfidf:", df_tfidf.shape)
print("Length of y_train:", len(y_train))

Shape of df_tfidf: (2741, 10000)
Length of y_train: 2741


In [72]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train, test_size=0.2, stratify=y_train,random_state=42)

In [73]:
x_train.shape

(2192, 10000)

In [74]:
x_val.shape

(549, 10000)

In [75]:
'''from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


model = MultinomialNB()
model.fit(x_train, y_train)'''
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

import numpy as np
from collections import Counter

# Count the number of users in each class
class_counts = Counter(y_train)

# Total number of samples
total_samples = len(y_train)

# Calculate class weights using log10
'''log_class_weights = {
    cls: np.log10(1 + total_samples / count)  # Logarithmic scaling of class weight
    for cls, count in class_counts.items()
}
'''
unique_classes, class_counts = np.unique(y_train, return_counts=True)
log_class_weights = {cls: 1 / (np.log(cnt) / np.log(20)) for cls, cnt in zip(unique_classes, class_counts)}
print("Class Weights with Log10 scaling:", log_class_weights)

class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(y_train), 
    y=y_train
)
class_weight_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}
print("Class Weights:", class_weight_dict)


from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

# Initialize SVM model with class weights
svm_model = LinearSVC(class_weight=log_class_weights,max_iter=2000,random_state=123)



#svm_model = LinearSVC(class_weight=class_weight_dict, max_iter=2000)
#non_linear_svm = SVC(kernel='rbf', class_weight=log_class_weights,max_iter=2000,random_state=42)
# Train the model
svm_model.fit(x_train, y_train)

Class Weights with Log10 scaling: {np.str_('art'): np.float64(0.5955211694024377), np.str_('entertainment'): np.float64(0.5394838964231858), np.str_('fashion'): np.float64(0.5470194853204887), np.str_('food'): np.float64(0.4981500114024804), np.str_('gaming'): np.float64(1.301029995663981), np.str_('health and lifestyle'): np.float64(0.49958412562650956), np.str_('mom and children'): np.float64(0.6268371758695794), np.str_('sports'): np.float64(0.6657464410787219), np.str_('tech'): np.float64(0.532667665109185), np.str_('travel'): np.float64(0.5487105689237657)}
Class Weights: {np.str_('art'): np.float64(1.4326797385620915), np.str_('entertainment'): np.float64(0.8496124031007752), np.str_('fashion'): np.float64(0.9171548117154812), np.str_('food'): np.float64(0.5359413202933986), np.str_('gaming'): np.float64(21.92), np.str_('health and lifestyle'): np.float64(0.545273631840796), np.str_('mom and children'): np.float64(1.842016806722689), np.str_('sports'): np.float64(2.43555555555555

LinearSVC(class_weight={np.str_('art'): np.float64(0.5955211694024377),
                        np.str_('entertainment'): np.float64(0.5394838964231858),
                        np.str_('fashion'): np.float64(0.5470194853204887),
                        np.str_('food'): np.float64(0.4981500114024804),
                        np.str_('gaming'): np.float64(1.301029995663981),
                        np.str_('health and lifestyle'): np.float64(0.49958412562650956),
                        np.str_('mom and children'): np.float64(0.6268371758695794),
                        np.str_('sports'): np.float64(0.6657464410787219),
                        np.str_('tech'): np.float64(0.532667665109185),
                        np.str_('travel'): np.float64(0.5487105689237657)},
          max_iter=2000, random_state=123)

In [76]:
'''from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Calculate class weights using log10
unique_classes, class_counts = np.unique(y_train, return_counts=True)
class_weights = {cls: 1 / np.log10(1 + cnt) for cls, cnt in zip(unique_classes, class_counts)}

print("Class Weights (log10 adjusted):", class_weights)

# Create Logistic Regression model
log_reg = LogisticRegression(
    multi_class='multinomial',  # for multiclass classification
    solver='lbfgs',            # efficient solver for multiclass problems
    class_weight=class_weights,  # handle class imbalance
    max_iter=1000               # allow sufficient iterations for convergence
)

# Train the model
log_reg.fit(x_train, y_train)

# Predict on validation set
y_val_pred = log_reg.predict(x_val)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_val, y_val_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
'''


'from sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import classification_report, confusion_matrix\nimport numpy as np\n\n# Calculate class weights using log10\nunique_classes, class_counts = np.unique(y_train, return_counts=True)\nclass_weights = {cls: 1 / np.log10(1 + cnt) for cls, cnt in zip(unique_classes, class_counts)}\n\nprint("Class Weights (log10 adjusted):", class_weights)\n\n# Create Logistic Regression model\nlog_reg = LogisticRegression(\n    multi_class=\'multinomial\',  # for multiclass classification\n    solver=\'lbfgs\',            # efficient solver for multiclass problems\n    class_weight=class_weights,  # handle class imbalance\n    max_iter=1000               # allow sufficient iterations for convergence\n)\n\n# Train the model\nlog_reg.fit(x_train, y_train)\n\n# Predict on validation set\ny_val_pred = log_reg.predict(x_val)\n\n# Evaluate the model\nprint("Classification Report:")\nprint(classification_report(y_val, y_val_pred))\n\n# Confus

In [77]:
'''from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(svm_model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)'''


"from sklearn.model_selection import GridSearchCV\nparam_grid = {'svc__C': [1, 5, 10, 50],\n              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}\ngrid = GridSearchCV(svm_model, param_grid)\n\n%time grid.fit(Xtrain, ytrain)\nprint(grid.best_params_)"

In [78]:
#@title Train Data
y_train_pred = svm_model.predict(x_train)

print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, zero_division=0))

Accuracy: 0.9904197080291971

Classification Report:
                      precision    recall  f1-score   support

                 art       1.00      0.98      0.99       153
       entertainment       1.00      0.99      0.99       258
             fashion       0.99      1.00      0.99       239
                food       0.97      1.00      0.99       409
              gaming       1.00      1.00      1.00        10
health and lifestyle       0.99      0.98      0.98       402
    mom and children       1.00      1.00      1.00       119
              sports       1.00      1.00      1.00        90
                tech       0.99      1.00      0.99       277
              travel       1.00      0.99      0.99       235

            accuracy                           0.99      2192
           macro avg       0.99      0.99      0.99      2192
        weighted avg       0.99      0.99      0.99      2192



In [79]:
y_val_pred = svm_model.predict(x_val)

print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))

Accuracy: 0.6539162112932605

Classification Report:
                      precision    recall  f1-score   support

                 art       0.33      0.11      0.16        38
       entertainment       0.49      0.35      0.41        65
             fashion       0.56      0.75      0.64        60
                food       0.81      0.90      0.86       102
              gaming       0.00      0.00      0.00         3
health and lifestyle       0.66      0.75      0.70       100
    mom and children       0.57      0.40      0.47        30
              sports       0.67      0.70      0.68        23
                tech       0.62      0.75      0.68        69
              travel       0.73      0.68      0.70        59

            accuracy                           0.65       549
           macro avg       0.54      0.54      0.53       549
        weighted avg       0.63      0.65      0.63       549

Confusion Matrix:
 [[ 4  4 11  3  0  6  1  2  5  2]
 [ 3 23  8  5  0 11  1  

In [80]:

from sklearn.feature_selection import chi2

chi2_scores, p_values = chi2(x_train, y_train)
feature_ranks = pd.DataFrame({
    "feature": feature_names,
    "chi2": chi2_scores
}).sort_values(by="chi2", ascending=False)

# Display top features
print(feature_ranks.head(50))

              feature       chi2
5857           mobile  70.038947
3151      galatasaray  64.372080
5417           lezzet  63.162402
4033          holiday  53.829798
973             bebek  47.689602
7593            stars  43.300991
8207          tiyatro  42.353641
3154           gaming  42.008584
4050            hotel  40.079955
5657              maç  38.747149
7943            tatil  37.348884
7284          seyahat  37.342196
1696            cloud  33.746323
2942          fashion  33.430562
8127              the  33.424664
2992             fiat  33.382606
6778         pırlanta  32.976403
3426              gol  32.642334
1498           burger  30.993859
9069            yemek  30.948702
1099         biletler  30.681831
1366           bodrum  29.656076
5434         lezzetli  29.295412
1084         beşiktaş  29.250379
335              anne  27.672113
498               art  27.346122
3123               fw  26.725727
8026        teknoloji  26.719875
979       bebeğinizin  26.571413
4902      

In [81]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_val, y_val_pred, labels=np.unique(y_train))
conf_matrix_df = pd.DataFrame(
    conf_matrix, index=np.unique(y_train), columns=np.unique(y_train)
)

print("Confusion Matrix:")
print(conf_matrix_df)

# Analyze specific misclassifications
print("Misclassified samples:")
for i, (true, pred) in enumerate(zip(y_val, y_val_pred)):
    if true != pred:
        print(f"Sample {i}: True={true}, Predicted={pred}")

Confusion Matrix:
                      art  entertainment  fashion  food  gaming  \
art                     4              4       11     3       0   
entertainment           3             23        8     5       0   
fashion                 2              3       45     2       0   
food                    0              5        0    92       0   
gaming                  0              0        1     0       0   
health and lifestyle    0              3        5     4       0   
mom and children        0              3        5     0       0   
sports                  0              0        1     1       0   
tech                    1              3        3     1       0   
travel                  2              3        1     5       0   

                      health and lifestyle  mom and children  sports  tech  \
art                                      6                 1       2     5   
entertainment                           11                 1       0     9   
fashion   

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create bigrams and trigrams
vectorizer = TfidfVectorizer(ngram_range=(1, 3))  # This will generate unigrams, bigrams, and trigrams
X_train_tfidf = vectorizer.fit_transform(x_train)

# Apply chi-squared test again on the new features
chi2_scores, p_values = chi2(X_train_tfidf, y_train)

# Create a dataframe of the features and their chi2 scores
feature_names = vectorizer.get_feature_names_out()
feature_ranks = pd.DataFrame({
    "feature": feature_names,
    "chi2": chi2_scores
}).sort_values(by="chi2", ascending=False)

# Display top features
print(feature_ranks.head(20))


ValueError: dimension mismatch

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix
cm = confusion_matrix(y_val, y_val_pred)

# Create a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [30]:
# Ensure that validation data is properly referenced
# Assuming you have a list of usernames and captions for the validation set
val_usernames = train_usernames[len(x_train):]  # Example placeholder
val_corpus = corpus[len(x_train):]  # Example placeholder

# Ensure y_val and y_val_pred are aligned with validation usernames
y_val_aligned = y_val[:len(val_usernames)]
y_val_pred_aligned = y_val_pred[:len(val_usernames)]

# Recreate the DataFrame for validation data
val_data = pd.DataFrame({
    "Username": val_usernames,
    "True Label": y_val_aligned,
    "Predicted Label": y_val_pred_aligned,
    "Captions": val_corpus
})

# Identify misclassified samples
misclassified = val_data[val_data["True Label"] != val_data["Predicted Label"]]

# Print misclassified samples with their captions
print("Misclassified Samples with Captions:")
for _, row in misclassified.iterrows():
    print(f"Username: {row['Username']}")
    print(f"True Label: {row['True Label']}")
    print(f"Predicted Label: {row['Predicted Label']}")
    print(f"Captions: {row['Captions']}\n")


Misclassified Samples with Captions:
Username: zekiinal09
True Label: art
Predicted Label: entertainment
Captions: bugün cabi muhtarım tayfun karaoğlan geyre muhtarım nevcet çoban tekeliler muhtarım serkan anil çamarasi muhtarım yusuf sevim yolalti muhtarım ali kocaağoğlu dikmen muhtarım hüseyin çeker yaykin muhtarım osman ali aktan ve güzelbeyli muhtarım celal ingay ile yaklaşan yerel seçimler ile ilgili sohbet ettik hoş sohbetlerinden dolayı kendilerine teşekkür ediyorum
birine çamur atmadan önce düşün ve sakin unutma ilk önce senin ellerin kirlenecek sağlikli mutlu bereketli bir hafta dileğiyle hayirli haftalar diliyorum
ilçemiz palamutçuk mahallesi şöför esnaflarımızdan ahmet kaya kardeşimizin oğlunun asker yemeğine katıldık allah kabul etsin sağ salim gidip gelmeyi nasip etsin inşaallah
bugün ilçemiz esnaf kefalet odası başkanı recep karataşziraat odası başkanı ibrahim iren şöförler odası başkanı mustafa gülgen ve iyi parti kadın kolları başkanı nihal deliktaşı ziyaret ettimkendil

In [ ]:
#@title Test Data
test_data_path = "/Users/Osama/Downloads/CS412PROJ/test-classification-round1.dat"

with open(test_data_path, "rt") as fh:
    for i, line in enumerate(fh):
        print(line.strip())
        if i >= 4:  # Stop after 5 lines
            break

print("*****")

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

ozhotelstr
elleturkiye
sozerinsaatorhangazi
sanliurfapiazzaavym
rusanozden
*****
['ozhotelstr', 'elleturkiye', 'sozerinsaatorhangazi', 'sanliurfapiazzaavym', 'rusanozden']


In [33]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)


test_unames.remove("screenname")

screenname


In [ ]:
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,abiye,abone,about,about the,...,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şık bir,şıklık,şıklığı,şımartın
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.007865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.024666,0.0,0.0,0.0,0.0,0.010521,0.0,0.0,0.0,0.0


In [35]:
'''def extract_features_for_regression(user_posts, user_profile):
    features = {}

    # Number of posts
    features['post_count'] = len(user_posts)

    # Number of followers
    features['follower_count'] = user_profile.get('follower_count', 0)

    # Whether the account is a business account
    features['is_business_account'] = int(user_profile.get('is_business_account', False))

    # Extract average like count
    like_counts = [post['like_count'] for post in user_posts if 'like_count' in post]
    features['avg_like_count'] = sum(like_counts) / len(like_counts) if like_counts else 0

    # Extract average comment count
    comment_counts = [post['comments_count'] for post in user_posts if 'comments_count' in post]
    features['avg_comment_count'] = sum(comment_counts) / len(comment_counts) if comment_counts else 0

    return features
'''

In [ ]:
'''from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_text(text: str):
    # Preprocess the text (lowercase, remove punctuation, etc.)
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text

# Extract features from captions using TF-IDF
corpus = []
for username, posts in username2posts_train.items():
    post_captions = [post['caption'] for post in posts if 'caption' in post]
    processed_captions = [preprocess_text(caption) for caption in post_captions]
    corpus.append(" ".join(processed_captions))

vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(corpus)'''

In [48]:
'''from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string

# Placeholder for the data - assuming you have username2posts and username2profile from your previous steps
features_list = []
for username, posts in username2posts_train.items():
    # Extract profile features
    profile = username2profile_train[username]
    profile_features = {
        'follower_count': profile.get('follower_count', 0),
        'is_business_account': int(profile.get('is_business_account', False)),
    }
    
    # Handle the case where there are no comments (i.e., empty posts)
    comment_counts = [post['comments_count'] for post in posts if 'comments_count' in post]
    profile_features['avg_comment_count'] = np.mean(comment_counts) if comment_counts else 0

    # Text-based features (Post captions using TF-IDF)
    post_captions = [str(post.get('caption', '')) for post in posts if 'caption' in post]
    caption_text = ' '.join(post_captions)  # Combine all captions
    profile_features['caption_length'] = len(caption_text.split())

    # Clean the caption: Remove punctuation and stopwords
    cleaned_caption = caption_text.translate(str.maketrans('', '', string.punctuation)).lower()
    words = cleaned_caption.split()
    
    # Remove stopwords
    valid_words = [word for word in words if word not in ENGLISH_STOP_WORDS]

    if valid_words:  # If there are valid words after removing stopwords
        tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Limit to 100 features
        tfidf_matrix = tfidf_vectorizer.fit_transform([caption_text])
        tfidf_features = tfidf_matrix.toarray().flatten()  # Convert the sparse matrix to an array
        
        # Add the TF-IDF features to the profile
        profile_features.update(dict(zip([f'tfidf_{i}' for i in range(len(tfidf_features))], tfidf_features)))
    else:
        # If no valid words after stopwords removal, set TF-IDF features to zero
        profile_features.update({f'tfidf_{i}': 0 for i in range(100)})
    
    # Combine all features into a single row for this user
    features_list.append(profile_features)

# Convert the list of feature dictionaries into a DataFrame
X = pd.DataFrame(features_list)

# Prepare target variable `y` (log-transformed like count)
like_counts = []
for username, posts in username2posts_train.items():
    like_count_list = [post.get('like_count', 0) for post in posts if 'like_count' in post]
    avg_like_count = np.mean(like_count_list) if like_count_list else 0
    like_counts.append(np.log10(avg_like_count + 1))  # Log-transform the like counts

y = pd.Series(like_counts)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model: RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model using MSE
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Convert back the predictions to the original like_count scale (by applying inverse log)
y_pred_orig = np.power(10, y_pred) - 1
y_test_orig = np.power(10, y_test) - 1

# Final output format
output = dict(zip(test_usernames, y_pred_orig))
print(output)
'''

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [86]:
# Prepare the target variable (like_count)
train_like_counts = []
for username, posts in username2posts_train.items():
    # Aggregate like_count for each user
    total_likes = sum(post.get("like_count", 0) or 0 for post in posts)  # Replace None with 0
    train_like_counts.append(total_likes)

# Apply log10 transformation to like_count
train_like_counts_log = np.log10(np.array(train_like_counts) + 1)  # Adding 1 to avoid log(0)

# Split data into training and validation sets
x_train_reg, x_val_reg, y_train_reg, y_val_reg = train_test_split(
    x_post_train, train_like_counts_log, test_size=0.2, random_state=42
)

# Initialize and train the regression model
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(x_train_reg, y_train_reg)

# Make predictions
y_val_pred_reg = regressor.predict(x_val_reg)

# Evaluate the model
mse = mean_squared_error(y_val_reg, y_val_pred_reg)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", np.sqrt(mse))


Mean Squared Error (MSE): 0.7147481609870759
Root Mean Squared Error (RMSE): 0.8454277976190965


In [6]:
test_pred = regressor.predict(df_test)

output = dict()
for index, uname in enumerate(test_unames):
  output[uname] = test_pred[index]

NameError: name 'regressor' is not defined

In [ ]:
with open("output.json", "w") as of:
  json.dump(output, of, indent=4)